# Implementation of the `scaredy_fish` model in Julia
### First step
Let's set up blank arrays that we will fill in, draw initial conditions, and set parameter values

In [149]:
## ---
## init
## ---
using Distributions, Distances, IterTools, BenchmarkTools

## -
## Define my landscape
## -

## number of rows and cols
nrows = 3
ncols = 8

## landscape is a binary matrix, where 1s are foraging patches (white tiles)
landscape = rand( Bernoulli( 0.5 ), nrows, ncols )

## indices for white tiles
whiteTiles = getindex.( findall( y -> y == 1, landscape ), [ 1 2 ] )

## -
## Define a bunch of parameters
## -

## number of agents
agents = 5

## time steps
time = 10

## walk step, this is the SD of the normal distribution from which step lengths are drawn
walk = 0.05

## run step, this is the SD of the normal distribution from which run lengths are drawn
run = 0.2

## sensory length scale
sensory = 2.5

## trigger time for false startle
trigger = 5

## -
## Define some empty arrays to fill in
## -

## empty 3D array to populate ( time x number of variables x agents )
r = zeros( time, 6, agents );

## populate first row of each matrix
for agent in 1 : size( r, 3 )
    
    ## put a random number in the first row
    r[ 1, 1, agent ] = Float64.( rand( Uniform( 0.5, 0.5 + size( landscape, 1 ) ) ) )
    r[ 1, 2, agent ] = Float64.( rand( Uniform( 0.5, 0.5 + size( landscape, 2 ) ) ) )
    
end

### Second step
We will loop through `times` and `agents` and do all the calculations

In [152]:
## ---
## Loop through time steps
## ---

## for t in ___time___
for t in 2 : time
    
    ## for a in agents
    for a in 1 : agents
        
        ## -
        ## first we need to know if we're in a white tile or black tile, and where the nearest white tile is
        ## -
        currX = r[ t - 1, 1, a ]
        currY = r[ t - 1, 2, a ]
        
        ## round them to find the nearest tile
        currPatchX = round.( Int, currX )
        currPatchY = round.( Int, currY )
        
        ## check that it is actually a tile, and you are not out of bounds
        if ( 0.5 < currPatchX < ( 0.5 + size( landscape, 1 ) ) ) && 
            ( 0.5 < currPatchY < ( 0.5 + size( landscape, 2 ) ) )
            
            ## if this is a white tile, we are already done
            if landscape[ currPatchX, currPatchY ] == 1
                
                ## make this my home patch
                homePatchX = currPatchX
                homePatchY = currPatchY
                
                ## record status in output
                r[ t, 4, a ] = landscape[ homePatchX, homePatchY ]
                r[ t, 5, a ] = homePatchX
                r[ t, 6, a ] = homePatchY
                
            end
                
            ## otherwise, find the nearest white tile
            if landscape[ currPatchX, currPatchY ] == 0
                
                ## distances to white tiles
                toWhiteTiles = colwise(Euclidean(), [1.1, 2.4], transpose( whiteTiles ) )
                
                ## which min
                homePatchX = whiteTiles[ argmin( ds ), 1 ]
                homePatchY = whiteTiles[ argmin( ds ), 2 ]
                
                ## record status in output
                r[ t, 4, a ] = landscape[ homePatchX, homePatchY ]
                r[ t, 5, a ] = homePatchX
                r[ t, 6, a ] = homePatchY
                
            end
            
        end 
            
        
        ## -
        ## Here starts the step updating section
        ## -
        
        ## if feeding
        if r[ t - 1, 3, a ] == 0
            
            ## random walk steps
            r[ t, 1, a ] = r[ t - 1, 1, a ] + rand( Normal( 0, walk ) )
            r[ t, 2, a ] = r[ t - 1, 2, a ] + rand( Normal( 0, walk ) )
            
        end
        
        ## if fleeing
        if r[ t - 1, 3, a ] == 1
            
            ## random walk steps
            r[ t, 1, a ] = r[ t - 1, 1, a ] + rand( Normal( 0, run ) )
            r[ t, 2, a ] = r[ t - 1, 2, a ] + rand( Normal( 0, run ) )
            
        end
        
    end
    
    ## -
    ## Here starts the information updating section
    ## -

    ## extract current coordinates
    C = hcat( r[ 2, 1, : ], r[ 2, 2, : ] )

    ## calculate distances between agents
    D = pairwise( Euclidean(), C; dims = 1 )

    ## threshold this matrix
    D[ D .> sensory ] .= 0

    ## convert to 1 / D ^ 2
    D[ D .> 0 ] = 1 ./ D[ D .> 0 ] .^ 2
    
    ## -
    ## trigger false alarm
    ## -
    if t == trigger
        
        ## switch state
        r[ t, 3, 1 ] = 1
        
    end

end

In [153]:
r

10×6×5 Array{Float64,3}:
[:, :, 1] =
 1.74495  3.98855  0.0  0.0  0.0  0.0
 1.62727  3.99145  0.0  1.0  1.0  2.0
 1.62813  4.04928  0.0  1.0  1.0  2.0
 1.54802  4.02361  0.0  1.0  1.0  2.0
 1.481    4.00924  1.0  1.0  1.0  2.0
 1.32809  3.8603   0.0  1.0  1.0  4.0
 1.33339  3.8193   0.0  1.0  1.0  4.0
 1.28898  3.78824  0.0  1.0  1.0  4.0
 1.29419  3.80042  0.0  1.0  1.0  4.0
 1.31511  3.7753   0.0  1.0  1.0  4.0

[:, :, 2] =
 0.599807  0.676164  0.0  0.0  0.0  0.0
 0.619396  0.663139  0.0  1.0  1.0  2.0
 0.658946  0.672182  0.0  1.0  1.0  2.0
 0.645277  0.704681  0.0  1.0  1.0  2.0
 0.556859  0.770546  0.0  1.0  1.0  2.0
 0.593056  0.767979  0.0  1.0  1.0  2.0
 0.597604  0.7839    0.0  1.0  1.0  2.0
 0.644061  0.796527  0.0  1.0  1.0  2.0
 0.633913  0.846429  0.0  1.0  1.0  2.0
 0.692113  0.91205   0.0  1.0  1.0  2.0

[:, :, 3] =
 3.3263   7.29105  0.0  0.0  0.0  0.0
 3.39009  7.21727  0.0  1.0  3.0  7.0
 3.49679  7.2167   0.0  1.0  3.0  7.0
 3.47651  7.15714  0.0  1.0  3.0  7.0
 3.45

In [128]:
vcat( transpose( [1.1, 2.4] ), getindex.( findall( y -> y == 1, landscape ), [ 1 2 ] ) )
pairwise( Euclidean(), x; dims = 1 )

24×24 Array{Float64,2}:
 0.0      1.41421  2.23607  2.0      …  3.0      3.16228  4.12311  5.09902
 1.41421  0.0      1.0      1.41421     2.23607  2.0      3.0      4.0
 2.23607  1.0      0.0      2.23607     2.82843  2.23607  3.16228  4.12311
 2.0      1.41421  2.23607  0.0         1.0      1.41421  2.23607  3.16228
 2.23607  1.0      1.41421  1.0         1.41421  1.0      2.0      3.0
 3.60555  2.23607  2.0      2.23607  …  2.0      1.0      1.41421  2.23607
 4.0      3.16228  3.60555  2.0         1.0      1.41421  1.0      1.41421
 4.47214  3.16228  3.0      2.82843     2.23607  1.41421  1.0      1.41421
 5.0      4.12311  4.47214  3.0         2.0      2.23607  1.41421  1.0
 5.38516  4.12311  4.0      3.60555     2.82843  2.23607  1.41421  1.0
 2.0      3.16228  3.60555  4.0      …  5.0      5.09902  6.08276  7.07107
 2.23607  3.0      3.16228  4.12311     5.09902  5.0      6.0      7.0
 2.82843  3.16228  3.0      4.47214     5.38516  5.09902  6.08276  7.07107
 1.0      2.23607  2.

In [148]:
print( whiteTiles )
ds = colwise(Euclidean(), [1.1, 2.4], transpose( whiteTiles ) )
whiteTiles[ argmin( ds ), : ]

[1 1; 2 1; 1 2; 2 2; 1 3; 2 3; 3 3; 1 4; 3 4; 2 5; 3 5; 3 6; 2 7; 3 7; 3 8]

2-element Array{Int64,1}:
 1
 2